In [4]:
from IPython.display import Latex
import numpy as np

# 常用numpy函数
## expand_dims
在指定位置插入新的轴来扩展数组形状

例如原本为一维的2个数据，axis=0，则shape变为(1,2),axis=1则shape变为(2,1)
再例如 原本为 (2,3),axis=0，则shape变为(1,2,3),axis=1则shape变为(2,1,3)

In [2]:
# 测试expand_dims函数
x = np.array([1,2,3])
print(x)
print("x.shape: ",x.shape)
y = np.expand_dims(x,axis=0)
print(y)
print("y.shape: ",y.shape)
print("y[0][1]: ",y[0][1])
y = np.expand_dims(x,axis=1)
print(y)
print("y.shape: ",y.shape)
print("y[1][0]: ",y[1][0])

NameError: name 'np' is not defined

## concatenate
根据指定的维度，对一个元组、列表中的list或者ndarray进行连接


In [8]:
# 例子，一个2*2的数组和一个1*2的数组，在第0维进行拼接，得到一个3*2的数组
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
c = np.concatenate((a, b), axis=0)
print(c)

# 一个2*2的数组和一个2*1的数组，在第1维进行拼接，得到一个2*3的数组：
c = np.concatenate((a, b.T), axis=1)
print(c)

[[1 2]
 [3 4]
 [5 6]]
[[1 2 5]
 [3 4 6]]


上面两个简单的例子中，拼接的维度的长度是不同的，但是其他维度的长度必须是相同的，这也是使用concatenate()函数的一个基本原则，违背此规则就会报错，例如一个2\*2的数组和一个1\*2的数组，在第1维进行拼接：

In [ ]:
# 以下代码会报错
np.concatenate((a, b), axis=1)

## stack
将数组的数据按照指定的维度进行堆叠

In [10]:
a = np.array([1,2,3])
b = np.array([2,3,4])
# 按行堆叠
print(np.stack([a,b],axis=0))
# 按列堆叠
print(np.stack([a,b],axis=1))

[[1 2 3]
 [2 3 4]]
[[1 2]
 [2 3]
 [3 4]]


可以看到，进行stack的两个数组必须有相同的形状，同时，输出的结果的维度是比输入的数组都要多一维的。我们拿第一个例子来举例，两个含3个数的一维数组在第0维进行堆叠，其过程等价于先给两个数组增加一个第0维，变为1\*3的数组，再在第0维进行concatenate()操作；第二个例子则是先将两个一维数组变为3*1的二维数组，再在第1维进行concatenate()操作

## vstack
等同于`stack(arrays,axis=0)`
## hstack
等同于`stack(arrays,axis=1)`

## linalg.pinv
求矩阵（通常是非方阵）的伪逆矩阵，X的伪逆矩阵即$(X^TX)^{-1}X^T$（易知$(X^TX)^{-1}X^TX=I$）

# 矩阵微积分
## 标量关于向量的偏导数
$$\frac{\partial y}{\partial \boldsymbol{x}}=[\frac{\partial y}{\partial x_1},\dots,\frac{\partial y}{\partial x_M}]^T$$

## 向量关于向量的偏导数
$$\frac{\partial f(\boldsymbol{x})}{\partial \boldsymbol{x}}=\begin{bmatrix} \frac{\partial y_1}{\partial x_1} & \dots  & \frac{\partial y_N}{\partial x_1} \\
\vdots & \vdots & \vdots \\
 \frac{\partial y_1}{\partial x_M} & \dots & \frac{\partial y_N}{\partial x_M}\end{bmatrix}$$

## 向量函数及其导数
$$\frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}} = \boldsymbol{I}（单位矩阵）$$
$$\frac{\partial \boldsymbol{Ax}}{\partial \boldsymbol{x}}=\boldsymbol{A}^T $$
$$\frac{\partial \boldsymbol{x^TA}}{\partial \boldsymbol{x}}=\boldsymbol{A} \\$$